In [1]:
CREATE DATABASE assignment5

Commands completed successfully.

Total execution time: 00:00:00.108

In [2]:
USE assignment5

Commands completed successfully.

Total execution time: 00:00:00.003

In [3]:
CREATE TABLE Students(
    id int primary key identity(1, 1),
    name varchar(50) not null
)

Commands completed successfully.

Total execution time: 00:00:00.024

In [4]:
CREATE TABLE Courses(
    id int primary key identity(1, 1),
    course varchar(50) not null
)

Commands completed successfully.

Total execution time: 00:00:00.012

In [5]:
CREATE TABLE StudentCourse(
    id int primary key identity(1, 1),
    sid int not null,
    constraint FK_Students foreign key(sid) references Students(id)
        on delete cascade,
    cid int not null,
    constraint FK_Courses foreign key(cid) references Courses(id)
        on delete cascade

)

Commands completed successfully.

Total execution time: 00:00:00.027

In [8]:
INSERT INTO Students(name) VALUES ('Robert'), ('Ariana'), ('Caleb'), ('Mihai'), ('Alex')

(5 rows affected)

Total execution time: 00:00:00.031

In [9]:
INSERT INTO Courses(course) VALUES('Databases'), ('Advanced Programming Methods'), ('Probability and Statistics')

(3 rows affected)

Total execution time: 00:00:00.010

In [17]:
SELECT * FROM Students
SELECT * FROM Courses

(5 rows affected)

(3 rows affected)

Total execution time: 00:00:00.009

id,name
1,Robert
2,Ariana
3,Caleb
4,Mihai
5,Alex


id,course
1,Databases
2,Advanced Programming Methods
3,Probability and Statistics


In [18]:
INSERT INTO StudentCourse(sid, cid) VALUES(1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 1), (4, 1), (4, 2), (4, 3), (5, 1) 

(10 rows affected)

Total execution time: 00:00:00.038

In [26]:
SELECT name, COUNT(*) nrCursuri FROM Students s
INNER JOIN StudentCourse sc ON s.id = sc.sid
INNER JOIN Courses c ON sc.cid = c.id
GROUP BY name

(5 rows affected)

Total execution time: 00:00:00.007

name,nrCursuri
Alex,1
Ariana,1
Caleb,2
Mihai,3
Robert,3


In [27]:
-- clustered index scan pentru ca nu avem niciun where
-- in sql, avem un singur clustered index pe tabel, acest index mentine o ordine fizica a elementelor
-- pentru ca putem avea doar o ordine fizica(logic) putem avea doar un clustered index
-- ca default behaviour, primary key-urile sunt clustered index pentru ca ele mentin aceasta ordine
-- clustered index scan : cand trebuie sa verfiicam toate records-urile din tabel pentru ca nu avem un filtru precum where care sa ne ajute
-- clustered index seek: cand avem un filtru de care sa ne folosim, deci vom avea o cautare mai rapida deoarece nu procesam fiecare tuple din baza de date

-- asta e un scan 
SELECT id, name 
FROM Students;

(5 rows affected)

Total execution time: 00:00:00.016

id,name
1,Robert
2,Ariana
3,Caleb
4,Mihai
5,Alex


In [29]:
-- clustered index seek: ne folosim de cheia primara dupa care filtram. daca ne foloseam de altceva gen WHERE name LIKE ... era tot scan.
SELECT * 
FROM Students 
WHERE id = 2;


(1 row affected)

Total execution time: 00:00:00.006

id,name
2,Ariana


In [30]:
CREATE NONCLUSTERED INDEX IX_Students_Name
ON Students(name);

Commands completed successfully.

Total execution time: 00:00:00.053

In [34]:
SELECT name 
FROM Students 
WHERE name = 'Robert'


(1 row affected)

Total execution time: 00:00:00.024

name
Robert


In [37]:
-- A Key Lookup happens when SQL Server is using an index (typically a non-clustered index) to find matching rows,
-- but it needs to go back to the clustered index (or main table) to retrieve additional columns 
-- that are not part of the non-clustered index.
ALTER TABLE Students ADD age int

Commands completed successfully.

Total execution time: 00:00:00.018

In [39]:
UPDATE Students SET age = 20 WHERE 1=1

(5 rows affected)

Total execution time: 00:00:00.028

In [41]:
UPDATE Students set age = 18 WHERE name IN ('Ariana', 'Caleb')

(2 rows affected)

Total execution time: 00:00:00.074

In [42]:
SELECT * FROM Students

(5 rows affected)

Total execution time: 00:00:00.005

id,name,age
1,Robert,20
2,Ariana,18
3,Caleb,18
4,Mihai,20
5,Alex,20


In [43]:
-- key lookup se uita in index pt nume si de acolo foloseste idu sa caute coloana in tabelu principal sa vada varsta
SELECT name, age
FROM Students
WHERE name = 'Ariana';


(1 row affected)

Total execution time: 00:00:00.018

name,age
Ariana,18


In [44]:
SELECT * FROM Courses WHERE course = 'Databases'
-- execution plan: nu am index, deci ma duc in fiecare entry din baza de date si caut pana gasesc una cu coloana course = databases

(1 row affected)

Total execution time: 00:00:00.078

id,course
1,Databases


In [45]:
CREATE NONCLUSTERED INDEX IX_Courses_Course
ON Courses(course);

Commands completed successfully.

Total execution time: 00:00:00.020

In [46]:
SELECT * FROM Courses WHERE course = 'Databases'
-- execution plan: am index, deci ma duc direct la coloana de databases fara sa mai caut altundeva si iau direct idu de acolo. O(1). inainte era O(n)

(1 row affected)

Total execution time: 00:00:00.017

id,course
1,Databases


In [47]:
CREATE VIEW StudentCoursesView AS
SELECT
    s.name AS StudentName,
    c.course AS CourseName
FROM
    Students s
JOIN
    StudentCourse sc ON s.id = sc.sid
JOIN
    Courses c ON sc.cid = c.id;


Commands completed successfully.

Total execution time: 00:00:00.032

In [48]:
SELECT * FROM StudentCoursesView

(10 rows affected)

Total execution time: 00:00:00.041

StudentName,CourseName
Robert,Databases
Robert,Advanced Programming Methods
Robert,Probability and Statistics
Ariana,Probability and Statistics
Caleb,Probability and Statistics
Caleb,Databases
Mihai,Databases
Mihai,Advanced Programming Methods
Mihai,Probability and Statistics
Alex,Databases
